**Aim: Write a program for the Implementation of a Generative Adversarial Network for generating synthetic shapes (like digits)**

In [38]:
# importing modules
from numpy import expand_dims, ones, zeros, random
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, LeakyReLU

In [39]:
# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1)):
    model = Sequential()
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [40]:
# define model
model = define_discriminator()
# summarize the model
model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_22 (Conv2D)              │ (None, 14, 14, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_22 (LeakyReLU)      │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 7, 7, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_23 (LeakyReLU)      │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_11 (Flatten)            │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │         3,137 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,705 (159.00 KB)

 Trainable params: 40,705 (159.00 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
# load and prepare mnist training images
def load_real_samples():
    (trainX, _), (_, _) = load_data()  # load mnist dataset
    X = expand_dims(trainX, axis=-1)  # expand to 3d, e.g. add channels dimension
    # convert from unsigned ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [0,1]
    X = X / 255.0
    return X

In [42]:
# select real samples
def generate_real_samples(dataset, n_samples):
    ix = random.randint(0, dataset.shape[0], n_samples)  # choose random instances
    # retrieve selected images
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return X, y

In [43]:
# generate n fake samples with class labels
def generate_fake_samples(n_samples):
  X = rand(28 * 28 * n_samples)  # generate uniform random numbers in [0,1]
  # reshape into a batch of grayscale images
  X = X.reshape((n_samples, 28, 28, 1))
  # generate 'fake' class labels (0)
  y = zeros((n_samples, 1))
  return X, y
# generate n fake samples with class labels
def generate_fake_samples(n_samples):
    # generate uniform random numbers in [0,1]
    X = random.rand(28 * 28 * n_samples)
    # reshape into a batch of grayscale images
    X = X.reshape((n_samples, 28, 28, 1))
    # generate 'fake' class labels (0)
    y = zeros((n_samples, 1))
    return X, y

In [44]:
# train the discriminator model
def train_discriminator(model, dataset, n_iter=100, n_batch=256):
  half_batch = int(n_batch / 2)
  # manually enumerate epochs
  for i in range(n_iter):
    X_real, y_real = generate_real_samples(dataset, half_batch) # get randomly selected 'real' samples
    # update discriminator on real samples
    _, real_acc = model.train_on_batch(X_real, y_real)
    # generate 'fake' examples
    X_fake, y_fake = generate_fake_samples(half_batch)
    # update discriminator on fake samples
    _, fake_acc = model.train_on_batch(X_fake, y_fake)
    # summarize performance
    print('>%d real=%.0f%% fake=%.0f%%' % (i+1, real_acc*100, fake_acc*100))

In [14]:
# define the discriminator model
model = define_discriminator()
# load image data
dataset = load_real_samples()
# fit the model
train_discriminator(model, dataset)

11490434/11490434 [==============================] - 0s 0us/step
>1 real=78% fake=5%
>2 real=88% fake=12%
>3 real=89% fake=32%
>4 real=89% fake=53%
>5 real=87% fake=69%
>6 real=87% fake=80%
>7 real=80% fake=91%
>8 real=88% fake=99%
>9 real=80% fake=99%
>10 real=77% fake=100%
>11 real=71% fake=100%
>12 real=74% fake=100%
>13 real=77% fake=100%
>14 real=75% fake=100%
>15 real=70% fake=100%
>16 real=77% fake=100%
>17 real=75% fake=100%
>18 real=70% fake=100%
>19 real=80% fake=100%
>20 real=73% fake=100%
>21 real=82% fake=100%
>22 real=88% fake=100%
>23 real=89% fake=100%
>24 real=90% fake=100%
>25 real=92% fake=100%
>26 real=95% fake=100%
>27 real=95% fake=100%
>28 real=94% fake=100%
>29 real=96% fake=100%
>30 real=98% fake=100%
>31 real=99% fake=100%
>32 real=98% fake=100%
>33 real=99% fake=100%
>34 real=100% fake=100%
>35 real=98% fake=100%
>36 real=99% fake=100%
>37 real=99% fake=100%
>38 real=100% fake=100%
>39 real=100% fake=100%
>40 real=99% fake=100%
>41 real=100% fake=100%
>42 rea